# This is Math Wizard

## Understanding DeepSeek Math with PyTorch and Transformers
## Start with installing all the necessary libraries

In [4]:
!pip install torch
!pip install transformers
!pip install accelerate
!pip install gradio

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

#Checking availability of CUDA (Compute Unified Device Architecture)

In [5]:
import torch

cuda_available = torch.cuda.is_available()
integrated_gpu_name = torch.cuda.get_device_name(0) if cuda_available else None

print("CUDA available:", cuda_available)
print("Integrated GPU name:", integrated_gpu_name)

CUDA available: True
Integrated GPU name: Tesla T4


## The AutoTokenizer function helps us convert human-readable text into a format that the model can understand. It's like translating words into a language the computer speaks.

## I am loading a pre-trained model called "DeepSeek-Math-7B-Instruct". This model specializes in understanding and generating mathematical text. We're specifying the data type (torch.bfloat16) for calculations and indicating that we want to use the GPU (device_map="cuda") for faster processing.

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-math-7b-instruct")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-math-7b-instruct",torch_dtype=torch.bfloat16, device_map="cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

## Here I configure the settings for text generation with the model.

## 1. I am loading the generation configuration from the pre-trained model "DeepSeek-Math-7B-Instruct".
## 2. I am Setting the pad_token_id to be the same as the eos_token_id.

In [7]:
# Configure generation settings
from transformers import GenerationConfig
model.generation_config = GenerationConfig.from_pretrained("deepseek-ai/deepseek-math-7b-instruct")
model.generation_config.pad_token_id = model.generation_config.eos_token_id

## Here I created solve_math_problem

1.  Input Handling:
    Converts single string inputs into a list.
2.  Solution Generation:
    For each question:
    1. Tokenizes the question.
    2. Generates a solution using the model.
    3. Decodes and appends the solution.
3. Output:
    Returns a list of solutions.

In [11]:
def solve_math_problem(questions):
    if isinstance(questions, str):  #if input is single string
        questions = [questions]

    results = []
    for question in questions:
        messages = [{"role": "user", "content": question}]
        input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
        outputs = model.generate(input_tensor.to(model.device), max_new_tokens=100)
        result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)
        results.append(result)
    return results

## Question and Answering

In [12]:
#Query 1
questions = "What is the square root of 16?"

results = solve_math_problem(questions)
print(results)

[' The square root of 16 is 4.']


In [15]:
#Query 2.1
questions = "How to solve Quadratic Equation ?"

result1 = solve_math_problem(questions)
print(results)

[' A quadratic equation is an equation that can be written as ax ² + bx + c = 0, where a ≠ 0.\n\nThe quadratic formula is derived from completing the square.\n\nThe quadratic formula is:\n\nx = (-b ± √(b² - 4ac)) / 2a\n\nThe term b² - 4ac is called the discriminant, because it can "discriminate" between the possible types of answer:\n']


In [16]:
#Query 2.2
questions = ["What to solve Quadratic Equation ?"]

result2 = solve_math_problem(questions)
print(results)

[' A quadratic equation is an equation that can be written as ax ² + bx + c = 0, where a ≠ 0.\n\nThe quadratic formula is derived from completing the square.\n\nThe quadratic formula is:\n\nx = (-b ± √(b² - 4ac)) / 2a\n\nThe term b² - 4ac is called the discriminant, because it can "discriminate" between the possible types of answer:\n']


In [17]:
#General Query

num_queries = int(input("How many queries do you want to solve? "))
queries = []
for i in range(num_queries):
    query = input(f"Enter query {i+1}: ")
    queries.append(query)

results = solve_math_problem(queries)
print("\nResults:")
for query, result in zip(queries, results):
    print(f"Query: {query}\nResult: {result}\n")

How many queries do you want to solve? 3
Enter query 1: what is 2+2?
Enter query 2: how to solve 4! ?
Enter query 3: what is 99999*99999 ?

Results:
Query: what is 2+2?
Result:  2+2=4

Query: how to solve 4! ?
Result:  4! means 4 factorial, which is the product of all positive integers up to 4.

4! = 4 × 3 × 2 × 1 = 24

So, 4! = 24.

Query: what is 99999*99999 ?
Result:  99999*99999 = 9999800001
The answer is $\boxed{9999800001}$.



## Math Wizad (User Interface with help of gradio )

In [19]:
import gradio as gr
interface = gr.Interface(
    fn=solve_math_problem,
    inputs="text",
    outputs="text",
    title="Math Wizard",
    description="""
        <h1>Welcome to the Math Wizard!</h1>
        <p>Ask any math question, and let the wizard guide you through the solution step-by-step.</p>
    """,
    allow_flagging=True,
    examples=[
        ["What is the integral of x^2?"],
        ["How do I solve a quadratic equation?"]
    ],
    theme="compact"
)
interface.launch()

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:776: UserWarning: Cannot load compact. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/compact (Request ID: Root=1-6650f34c-616dad67116af79d0552980a;7bab18bd-fec6-4db3-be9c-06d342393c03)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")
/usr/local/lib/python3.10/dist-packages/gradio/interface.py:373: UserWarning: The `allow_flagging` parameter in `Interface` nowtakes a string value ('auto', 'manual', or 'never'), not a boolean. Setting parameter to: 'manual'.
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1546f14940d81b62b9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
